# Final Exam - Data Analytics with Apache Spark

## Rules
1. Do not rename the placeholder variables
2. The required results / outputs must be assigned to corresponding placeholder variables. Otherwise, no points will be counted.
3. Variables prefixed with `df_` have DataFrames datatype. You must return or assign DataFrames value to those variables.

## Init codes - DO NOT CHANGE

##### Init Spark Session for the Exam

In [ ]:
from pyspark.sql import SparkSession
# the following Spark session creation contains jars which support writing to Delta format
# note: checkout this link to see which version of Delta is compatible with your version of Pyspark: https://docs.delta.io/latest/releases.html
spark = SparkSession.builder \
    .appName("Final Exam Application")  \
    .config("spark.sql.shuffle.partitions", "2")    \
    .config("spark.sql.streaming.forceDeleteTempCheckpointLocation", "true")    \
    .config("spark.sql.streaming.checkpointLocation", "final_exam/streaming_checkpoints")    \
	.config('spark.jars.packages', 'io.delta:delta-spark_2.12:3.2.0,graphframes:graphframes:0.8.4-spark3.5-s_2.12')	\
	.config('spark.sql.extensions', 'io.delta.sql.DeltaSparkSessionExtension')	\
	.config('spark.sql.catalog.spark_catalog', 'org.apache.spark.sql.delta.catalog.DeltaCatalog')	\
    .getOrCreate()

### Exam Evaluation Helper

In [ ]:
############### MUST EXECUTE && DO NOT CHANGE  ANY ###############
import hashlib
def __eval_res(var_name: str, val) -> str:
	if val:
		hash_val = hashlib.sha256(str(val).encode("utf-8")).hexdigest()
	else:
		hash_val = None
	print(f"__evaluated:{var_name}:{hash_val}")

## Data Engineering Tasks

In [ ]:
input_ml_path = "final_exam/rename_me_to_your_id/data/input/ml"
input_stream_path = "final_exam/rename_me_to_your_id/data/input/stream"
input_graph_path = "final_exam/rename_me_to_your_id/data/input/graph"


### Question 1: DataFrame Creation


**Car Evaluation Dataset Explanation**

The Car Evaluation dataset consists of various features that describe different aspects of cars, along with a target variable indicating their evaluation level. Below is a detailed explanation of each feature in the dataset.

**Features**

| Variable Name        | Type        | Description                                                                                     |
|----------------------|-------------|-------------------------------------------------------------------------------------------------|
| **buying**           | Categorical | Represents the buying price of the car, categorized as `vhigh`, `high`, `med`, or `low`.      |
| **maint**            | Categorical | Indicates the maintenance price of the car, categorized as `vhigh`, `high`, `med`, or `low`.  |
| **doors**            | Categorical | The number of doors in the car, which can be `2`, `3`, `4`, or `5more`.                       |
| **persons**          | Categorical | Capacity in terms of persons to carry, categorized as `2`, `4`, or `more`.                    |
| **lug_boot**         | Categorical | The size of the luggage boot, categorized as `small`, `med`, or `big`.                        |
| **safety**           | Categorical | Estimated safety of the car, categorized as `low`, `med`, or `high`.                          |
| **made_year**        | Numeric     | The year the car was manufactured, represented as an integer.                                  |
| **engine_size**      | Numeric     | The size of the engine in liters, represented as a double.                                     |
| **horsepower**       | Numeric     | The horsepower of the car, represented as an integer.                                         |
| **fuel_type**        | Categorical | The type of fuel used by the car, which can be `Petrol`, `Diesel`, or `Electric`.             |
| **price_score**      | Numeric     | A numeric score based on the buying price categories (1 for low to 4 for vhigh).              |
| **maintenance_score**| Numeric     | A numeric score based on maintenance price categories (1 for low to 4 for vhigh).             |
| **safety_score**     | Numeric     | A numeric score based on safety levels (1 for low to 3 for high).                             |
| **capacity_score**   | Numeric     | A numeric score based on capacity indicated in persons (1 for 2 persons to 3 for more).       |
| **class**            | Target      | The evaluation level of the car, which can be `unacc` (unacceptable), `acc` (acceptable), `good`, or `vgood` (very good). |


**Summary**

This dataset is designed to facilitate machine learning tasks related to car evaluations. Each feature provides valuable information that can help in predicting the evaluation level of a car based on its attributes.


**Requirements**
- Create a dataframes named `df_car` from `data/input/ml/car_eval.csv`
- The csv file does not contain headers, you need to define it. *Make sure the column names, order of columns and the data types meet the below requirements*:

**Data schema**
```text
root
 |-- buying: string (nullable = true)
 |-- maint: string (nullable = true)
 |-- doors: string (nullable = true)
 |-- persons: string (nullable = true)
 |-- lug_boot: string (nullable = true)
 |-- safety: string (nullable = true)
 |-- price_score: integer (nullable = true)
 |-- maintenance_score: integer (nullable = true)
 |-- safety_score: integer (nullable = true)
 |-- capacity_score: integer (nullable = true)
 |-- made_year: integer (nullable = true)
 |-- engine_size: double (nullable = true)
 |-- horsepower: integer (nullable = true)
 |-- fuel_type: string (nullable = true)
 |-- class: string (nullable = true)
 ```

In [ ]:
# Read the CSV file into a DataFrame
car_data_file_path = f"{input_ml_path}/car_eval.csv"

df_car = spark.read.___

In [ ]:
df_car.show(10)

In [ ]:
############### MUST EXECUTE && DO NOT CHANGE ANY ###############
__eval_res('__eval_01_01', df_car.schema)

In [ ]:
############### MUST EXECUTE && DO NOT CHANGE ANY ###############
__eval_res('__eval_01_02', df_car.count())

### Question 2: Data Cleaning
In `df_car`, some entries are null. Handle them with following requirements:

1. Drop all rows which have `class` is null. Place the result into new dataframes: `df_car_no_class_null`

In [ ]:
df_car_no_class_null = ___

In [ ]:
############### MUST EXECUTE && DO NOT CHANGE ANY ###############
__eval_res('__eval_02_01', df_car_no_class_null.count())

2. Impute the missing values for numeric columns in `df_car_no_class_null` using `pyspark.ml.feature.Imputer`:
  - `made_year` -> using **mode** value
  - `engine_size` -> using **median** value
  - `horsepower`  -> using **mean** value

Output imputed data to dataframes: `df_imputed` for the next steps.

In [ ]:
df_imputed = ___

In [ ]:
############### MUST EXECUTE && DO NOT CHANGE ANY ###############
__eval_res('__eval_02_02', df_imputed.na.drop(subset=['made_year', 'engine_size', 'horsepower']).count())

## Feature Engineering Tasks

### Question 3: Encoding Categorical Variables
Convert a categorical variable (`fuel_type`) into numerical format using `pyspark.ml.feature.StringIndexer`:
- Input column: `fuel_type`
- Output column: `fuel_type_index`
- Input dataframes: `df_imputed`
- Output dataframes: `df_encoded`

In [ ]:
df_encoded = ___
df_encoded.show(10)

In [ ]:
############### MUST EXECUTE && DO NOT CHANGE ANY ###############
__eval_res('__eval_03_01', df_encoded.select('fuel_type_index'))

In [ ]:
############### MUST EXECUTE && DO NOT CHANGE ANY ###############
__eval_res('__eval_03_02', df_encoded.selectExpr('sum(fuel_type_index)').collect())

### Question 4: Creating New Feature
- From the year of manufacture (`made_year`), create a new feature named `car_age`
- Formula: ```car_age = current_year - made_year```
- Input dataframes: `df_encoded`
- Output dataframes: `df_car_with_new_feature`

In [ ]:
df_car_with_new_feature = ___

In [ ]:
df_car_with_new_feature.show(10)

In [ ]:
############### MUST EXECUTE && DO NOT CHANGE ANY ###############
__eval_res('__eval_04_01', df_car_with_new_feature.select('car_age'))

In [ ]:
############### MUST EXECUTE && DO NOT CHANGE ANY ###############
__eval_res('__eval_04_02', df_car_with_new_feature.selectExpr('avg(car_age)').collect())

### Question 5: Feature Selection
You now have a DataFrame `df_car_with_new_feature` with multiple features. Select only the numerical features for the next step:
- Algorithm: `pyspark.ml.feature.VectorAssembler`
- Input dataframes: `df_car_with_new_feature`
- Input column: all the columns which have numeric data types (must be exhaustive)
- Output column: `features`
- Output dataframes: `df_selected_features`

In [ ]:
df_selected_features = ___
df_selected_features.show(10)

In [ ]:
############### MUST EXECUTE && DO NOT CHANGE ANY ###############
__eval_res('__eval_05_01', df_selected_features.select('features'))

In [ ]:
############### MUST EXECUTE && DO NOT CHANGE ANY ###############
__eval_res('__eval_05_02', len(df_selected_features.select('features').take(1)[0][0]))

### Question 6: Normalization of Features
Normalize / standardize numerical features to ensure they contribute equally to the model training process:
- Algorithm: `pyspark.ml.feature.StandardScaler`
- Algorithm config: `withMean=True`, `withStd=True`
- Input dataframes: `df_selected_features`
- Input column: `features`
- Output dataframes: `df_scaled`
- Output column: `features_scaled`

In [ ]:
df_scaled = ___
df_scaled.show(10)

In [ ]:
############### MUST EXECUTE && DO NOT CHANGE ANY ###############
__eval_res('__eval_06_01', df_scaled.select('features_scaled'))

In [ ]:
############### MUST EXECUTE && DO NOT CHANGE ANY ###############
__eval_res('__eval_06_02', len(df_scaled.select('features_scaled').take(1)[0][0]))

## Train New Models, Evaluate Models, Select Model Performance

### Question 7: Preparing Training and Testing Datasets

**Detailed requirements** (step by step):

Index the categorical label
- Source dataframes: `df_scaled`
- Using `StringIndexer` to index the label column `class`. Named the output column as `class_index`.
- The dataframes with indexed label is named: `df_label_indexed`

Split data
- Training dataset (dataframes): weight = 80% - namely `df_train`
- Testing dataset (dataframes): weight = 20% - namely `df_test`
- Algorithm: `randomSplit` with `seed` must be configured as `100`




In [ ]:
df_label_indexed = ___

In [ ]:
df_label_indexed.show(10)

In [ ]:
# Split Dataset into Training and Test Sets
df_train, df_test = df_label_indexed.___


In [ ]:
df_train.show(5)

In [ ]:
df_test.show(5)

In [ ]:
############### MUST EXECUTE && DO NOT CHANGE ANY ###############
__eval_res('__eval_07_01', df_train.count())

In [ ]:
############### MUST EXECUTE && DO NOT CHANGE ANY ###############
__eval_res('__eval_07_02', df_test.count())

### Question 8: Training a New Model
Train a model using the DataFrame `df_train`:
- Algorithm: `pyspark.ml.classification.RandomForestClassifier`
- Model output: `rf_model`

In [ ]:
rf_model = ___

In [ ]:
############### MUST EXECUTE && DO NOT CHANGE ANY ###############
__eval_res('__eval_08_01', rf_model.numClasses)

In [ ]:
############### MUST EXECUTE && DO NOT CHANGE ANY ###############
__eval_res('__eval_08_02', rf_model.numFeatures)

### Question 9: Model Evaluation Metrics
Evaluate your trained model using `accuracy` on test data.

Requirements:
1. `df_predictions`: predictions of test data using the trained model
2. `accuracy` value obtained from evaluation

In [ ]:
# Make Predictions
df_predictions = ___

In [ ]:
accuracy = ___

# Show predictions
df_predictions = ___


In [ ]:
############### MUST EXECUTE && DO NOT CHANGE ANY ###############
__eval_res('__eval_09_01', df_predictions.select("features_scaled", "class_index", "prediction"))

In [ ]:
############### MUST EXECUTE && DO NOT CHANGE ANY ###############
__eval_res('__eval_09_02', accuracy)

### Question 10: Cross-Validation for Hyperparameter Tuning
- Set up cross-validation for hyperparameter tuning of your model using a parameter grid (`paramGrid`): `pyspark.ml.tuning.ParamGridBuilder` 
- Display best parameters after fitting
- Get the best model as `best_model` for the next steps

In [ ]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [ ]:
paramGrid = ___
crossval = ___
cv_model = ___

In [ ]:
# Get Best Model and Parameters
best_model = ___
print("Best Model Parameters:")
print("Num Trees:", best_model.getNumTrees)
print("Max Depth:", best_model.getMaxDepth)

In [ ]:
############### MUST EXECUTE && DO NOT CHANGE ANY ###############
__eval_res('__eval_10_01', best_model.numTrees)

In [ ]:
############### MUST EXECUTE && DO NOT CHANGE ANY ###############
__eval_res('__eval_10_02', best_model.numClasses)

In [ ]:
############### MUST EXECUTE && DO NOT CHANGE ANY ###############
__eval_res('__eval_10_03', best_model.numFeatures)

### Question 11: Predictions Using the Selected (Best) Model
- Using your best-performing model from previous steps, make predictions on new data stored in `unlabeled_car.csv` (no headers). 
- The predicted label column is named as `prediction`
- Display predictions in a new dataframes: `df_predicted_unlabeled`
- Create a new column named `original_class` which have the actual class values (`acc, good, unacc`) rather than indexed values (`1.0, 2.0`). Suggestion: Use `pyspark.ml.feature.IndexToString`


In [ ]:
# Read the CSV file into a DataFrame
file_path_car_unlabeled_data = f"{input_ml_path}/unlabeled_car.csv"

df_unlabeled_car = spark.read.___

In [ ]:
# Add processing steps here, to make the model able to transform the unlabeled data

In [ ]:
df_predicted_unlabeled = best_model.transform(df_unlabeled_car)
df_predicted_unlabeled.select("prediction").show()

In [ ]:
# Create a new column named `original_class` which have the actual class values (`acc, good, unacc`) rather than indexed values (`1.0, 2.0`). Suggestion: Use `pyspark.ml.feature.IndexToString`
# Input codes here
converter = ___
df_converted = converter.transform(df_predicted_unlabeled)

In [ ]:
df_converted.select('prediction', 'original_class').show()

In [ ]:
############### MUST EXECUTE && DO NOT CHANGE ANY ###############
__eval_res('__eval_11_01', df_predicted_unlabeled.select("prediction"))

In [ ]:
############### MUST EXECUTE && DO NOT CHANGE ANY ###############
__eval_res('__eval_11_02', df_predicted_unlabeled.select("prediction"))

In [ ]:
############### MUST EXECUTE && DO NOT CHANGE ANY ###############
__eval_res('__eval_11_02', df_converted.select('prediction', 'original_class'))

## Spark Structured Streaming

### Question 12: Read and Write Stream
- Set up a basic streaming job that reads data from a CSV source (no headers) 
- Trigger mode: `availableNow`
- Writes it to a memory table named `tbl_streaming_01`
- Output mode is set to `update`
- Filter condition: `made_year > 2015`
- Print the results using Spark SQL
- Stop the Streaming query

In [ ]:
# For Structured Streaming
spark.conf.set("spark.sql.streaming.checkpointLocation", "final_exam/streaming_checkpoints")

In [ ]:
stream_path = "final_exam/rename_me_to_your_id/data/input/stream"

In [ ]:
# Define the schema
unlabeled_car_schema = ___

In [ ]:
df_streaming_data = spark.___

In [ ]:
query = df_streaming_data.___

In [ ]:
import time
time.sleep(5) # wait few seconds for the streaming query to be fully started

df_tbl_streaming_01 = spark.sql(___)

df_tbl_streaming_01.show()

In [ ]:
############### MUST EXECUTE && DO NOT CHANGE ANY ###############
__eval_res('__eval_12_01', df_tbl_streaming_01.count())

In [ ]:
query.stop()

### Question 13: Writing Streaming Output to File Sink
- Write streaming output of `df_streaming_data` file sink in Parquet format.
- Trigger mode is default
- Output mode is "Append"
- Stop the query after use
- Read to the data frame named `df_written_stream`
- Show top 10 rows of the dataframe `df_written_stream`

In [ ]:
stream_output_path = "final_exam/rename_me_to_your_id/data/output/stream"

In [ ]:
query_file_sink = df_streaming_data.___

In [ ]:
query_file_sink.stop()

In [ ]:
df_written_stream = spark.read.___
df_written_stream.show(10)

In [ ]:
############### MUST EXECUTE && DO NOT CHANGE ANY ###############
__eval_res('__eval_13_01', df_written_stream.count())

## Graph Analytics with GraphFrames

### Question 14: Creating GraphFrames from DataFrames
- Create a GraphFrame using vertices and edges stored in two separate DataFrames named `vertices` and `edges`. 
- Display basic information about the graph created.


In [ ]:
# %pip install graphframes

In [ ]:
# config for Graph
spark.sparkContext.setCheckpointDir('final_exam/spark_checkpoints')

In [ ]:
from graphframes import GraphFrame

v_path = 'final_exam/rename_me_to_your_id/data/input/graph/vertices.csv'
e_path = 'final_exam/rename_me_to_your_id/data/input/graph/edges.csv'

df_vertices = spark.___
df_edges = spark.___

In [ ]:
g = ___

In [ ]:
print(g.vertices.show())
print(g.edges.show())

In [ ]:
############### MUST EXECUTE && DO NOT CHANGE ANY ###############
__eval_res('__eval_14_01', g.vertices.count())

In [ ]:
############### MUST EXECUTE && DO NOT CHANGE ANY ###############
__eval_res('__eval_14_02', g.edges.count())

### Question 15: Finding Shortest Paths in Graphs
Using your GraphFrame g, find shortest paths from vertex ID '6' to all other vertices within a maximum path length of 3. Display results as paths found.


In [ ]:
results = g.___
results.show()

In [ ]:
############### MUST EXECUTE && DO NOT CHANGE ANY ###############
__eval_res('__eval_15_01', results.count())

### Question 16: PageRank Algorithm Implementation
Implement PageRank algorithm on your GraphFrame to find important vertices based on their connectivity.

In [ ]:
results_pagerank = g.___
results_pagerank.vertices.select("id", "pagerank").show()

In [ ]:
############### MUST EXECUTE && DO NOT CHANGE ANY ###############
__eval_res('__eval_16_01', results_pagerank.vertices.select("id", "pagerank"))

In [ ]:
############### MUST EXECUTE && DO NOT CHANGE ANY ###############
__eval_res('__eval_16_02', results_pagerank.vertices.count())

### Question 17: Connected Components Analysis
Perform connected components analysis on your GraphFrame to identify clusters of connected vertices.


In [ ]:
result_cc = g.___
result_cc.select("id", "component").show()

In [ ]:
############### MUST EXECUTE && DO NOT CHANGE ANY ###############
__eval_res('__eval_17_01', result_cc.select("id", "component"))

In [ ]:
############### MUST EXECUTE && DO NOT CHANGE ANY ###############
__eval_res('__eval_17_01', result_cc.select("id", "component").count())

### Question 18: Triangle Count in Graphs
Count triangles formed by vertices in your GraphFrame using triangle counting functionality.


In [ ]:
triangle_count_result = g.___
triangle_count_result.select("id", "count").show()

In [ ]:
############### MUST EXECUTE && DO NOT CHANGE ANY ###############
__eval_res('__eval_18_01', triangle_count_result.select("id", "count"))

In [ ]:
############### MUST EXECUTE && DO NOT CHANGE ANY ###############
__eval_res('__eval_18_02', triangle_count_result.select("id", "count").count())

## Complete & Cleanup

In [ ]:
spark.stop()

In [ ]:
### Clean up Checkpoints & Outputs
import os
import shutil

# Define the path to the checkpoint directory
spark_checkpoint_dir = "final_exam/spark_checkpoint/"
stream_checkpoint_dir = "final_exam/streaming_checkpoints/"
directories = [spark_checkpoint_dir, stream_checkpoint_dir]

# Remove the checkpoint directory if it exists
for dir in directories:
	if os.path.exists(dir):
		shutil.rmtree(dir)
		print(f"Directory {dir} cleaned up.")
	else:
		print(f"Directory {dir} does not exist.")